Device tree overlay

In [ ]:
import os

class overlay (object):

    def __init__ (self, overlay:str):
        if not isinstance(overlay, str):
            raise TypeError("Bitstream name has to be a string.")

        if os.path.isfile("{}.dts".format(overlay)):
            self.overlay = overlay
        else:
            raise IOError('Device tree overlay {}.dts does not exist.'.format(overlay))

        os.system("dtc -I dts -O dtb -o {overlay}.dtbo -@ {overlay}.dts".format(overlay=overlay))
        os.system('cat /opt/redpitaya/fpga/{}/fpga.bit > /dev/xdevcfg'.format(overlay))
        os.system("mkdir /sys/kernel/config/device-tree/overlays/{}".format(overlay))
        os.system("cat {overlay}.dtbo > /sys/kernel/config/device-tree/overlays/{overlay}/dtbo".format(overlay=overlay))

    def __del__ (self):
        os.system("rmdir /sys/kernel/config/device-tree/overlays/{}".format(overlay))
        self.overlay = None

In [ ]:
o = overlay("mercury")

In [ ]:
import os
import mmap
import numpy as np

class hwid ():

    regset_dtype = np.dtype([
        ('hwid' , 'uint32'),
        ('rsv0' , 'uint32'),  # reserved
        ('efuse', 'uint32'),
        ('rsv1' , 'uint32'),  # reserved
        ('dna'  , 'uint32', 2),
        ('rsv3' , 'uint32', 2),  # reserved
        ('gith' , 'uint32', 5)
    ])

    def __init__ (self, uio:str = '/dev/uio/hwid'):
        # TODO add exclusive lock
        try:
            self.uio_dev = os.open(uio, os.O_RDWR | os.O_SYNC)
        except OSError as e:
            raise IOError(e.errno, "Opening {}: {}".format(uio, e.strerror))

        try:
            self.uio_mem = mmap.mmap(
                fileno=self.uio_dev, length=mmap.PAGESIZE, offset=0x0,
                flags=mmap.MAP_SHARED, prot=(mmap.PROT_READ | mmap.PROT_WRITE))
        except OSError as e:
            raise IOError(e.errno, "Mapping {}: {}".format(uio, e.strerror))

        regset_array = np.recarray(1, self.regset_dtype, buf=self.uio_mem)
        self.regset = regset_array[0]
        
    def __del__ (self):
        self.uio_mem.close()
        try:
            os.close(self.uio_dev)
        except OSError as e:
            raise IOError(e.errno, "Closing {}: {}".format(uio, e.strerror))

In [ ]:
regid = hwid();

In [ ]:
del(regid)

In [ ]:
regid.regset

In [ ]:
class asg ():
    
    FS = 125000000.0
    # control register masks
    CTL_STO_MASK = (1<<3) # 1 - stop/abort; returns 1 when stopped
    CTL_STA_MASK = (1<<2) # 1 - start
    CTL_SWT_MASK = (1<<1) # 1 - sw trigger bit (sw trigger must be enabled)
    CTL_RST_MASK = (1<<0) # 1 - reset state machine so that it is in known state
    # linear addition multiplication register width
    DWM = 14
    DWS = 14

    regset_dtype = np.dtype([
        # control/status
        ('ctl_sts', 'uint32'),
        # trigger configuration
        ('cfg_trg', 'uint32'),  # trigger mask
        ('rsv0'   , 'uint32', 2),  # reserved
        # buffer configuration
        ('cfg_siz', 'uint32'),  # size
        ('cfg_off', 'uint32'),  # offset
        ('cfg_stp', 'uint32'),  # step
        ('rsv1'   , 'uint32'),  # reserved
        # burst mode
        ('cfg_bmd', 'uint32'),  # mode [1:0] = [inf, ben]
        ('cfg_bdl', 'uint32'),  # data length
        ('cfg_bln', 'uint32'),  # length (data+pause)
        ('cfg_bnm', 'uint32'),  # number of bursts pulses
        # burst status
        ('sts_bln', 'uint32'),  # length (current position inside burst length)
        ('sts_bnm', 'uint32'),  # number (current burst counter)
        # linear transformation
        ('cfg_mul',  'int32'),  # multiplier (amplitude)
        ('cfg_sum',  'int32')   # addedr (offset)
    ])

    def __init__ (self, index:int, uio:str = '/dev/uio/asg'):
        uio = uio+str(index)
        # TODO add exclusive lock
        try:
            self.uio_dev = os.open(uio, os.O_RDWR | os.O_SYNC)
        except OSError as e:
            raise IOError(e.errno, "Opening {}: {}".format(uio, e.strerror))

        try:
            self.uio_mem = mmap.mmap(
                fileno=self.uio_dev, length=mmap.PAGESIZE, offset=0x0,
                flags=mmap.MAP_SHARED, prot=(mmap.PROT_READ | mmap.PROT_WRITE))
        except OSError as e:
            raise IOError(e.errno, "Mapping {}: {}".format(uio, e.strerror))

        regset_array = np.recarray(1, self.regset_dtype, buf=self.uio_mem)
        self.regset = regset_array[0]
        
    def __del__ (self):
        self.uio_mem.close()
        try:
            os.close(self.uio_dev)
        except OSError as e:
            raise IOError(e.errno, "Closing {}: {}".format(uio, e.strerror))

    def reset (self):
        # reset state machine
        self.regset.ctl_sts = self.CTL_RST_MASK

    def run (self):
        # start state machine
        self.regset.ctl_sts = self.CTL_STA_MASK

    def run_status (self) -> bool:
        # start state machine
        return bool(self.regset.ctl_sts & self.CTL_STA_MASK)

    def stop (self):
        # stop state machine
        self.regset.ctl_sts = self.CTL_STO_MASK

    def trigger (self):
        # activate SW trigger
        self.regset.ctl_sts = self.CTL_SWT_MASK

    @property
    def amplitude (self) -> float:
        return (self.regset.cfg_mul / (1 << self.DWM))
    
    @setter
    def amplitude (self, value: float):
        # TODO check range
        self.regset.cfg_mul = value * (1 << self.DWM))
    
    @property
    def offset (self) -> float:
        return (self.regset.cfg_sum / (1 << self.DWS))
    
    @setter
    def offset (self, value: float):
        # TODO check range
        self.regset.cfg_sum = value * (1 << self.DWS))

    @property
    def frequency (self) -> float:
        siz = self.regset.cfg_siz + 1
        stp = self.regset.cfg_stp + 1
        return (stp / siz * self.FS)
    
    @setter
    def frequency (self, value: float):
        # TODO add range check
        siz = self.regset.cfg_siz + 1
        self.regset.cfg_stp = int(siz * (value / self.FS)) - 1
    
    @property
    def phase (self) -> float:
        siz = self.regset.cfg_siz + 1
        off = self.regset.cfg_off
        return (stp / siz * 360)
    
    @setter
    def phase (self, value: float):
        # TODO add range check
        siz = self.regset.cfg_siz + 1
        self.regset.cfg_off = int(siz * (value % 360) / 360)

    @property
    def waveform (self):
        siz = (self.regset.cfg_siz + 1) >> self.CWF
        # TODO: nparray
        return [self.table[i] / (1 << self.DWS) for i in range(siz)]
    
    @setter
    def waveform (self, value):
        # TODO check table size shape
        siz = size(value)
        for i in range(siz):
            # TODO add saturation
            self.table[i] = int(value[i] * (1 << self.DWS))
        self.regset.cfg_siz = (siz << self.CWF) - 1

    import ctype
    from enum import Enum
    class modes(Enum):
        CONTINUOUS = c_int(0x0)
        BURST_FIN  = c_int(0x1)
        BURST_INF  = c_int(0x3)
    
    @property
    def mode (self):
        return (modes(self.regset.cfg_bst))

    @setter
    def mode (self, value):
        # TODO check range
        self.regset.cfg_bst = value

    @property
    def burst_repetitions (self) -> int:
        return (self.regset.cfg_bnm)

    @setter
    def burst_repetitions (self, value: int):
        # TODO check range
        self.regset.cfg_bnm = value
        
    @property
    def burst_data_len (self) -> int:
        return (self.regset.cfg_bdl)

    @setter
    def burst_data_len (self, value: int):
        # TODO check range
        self.regset.cfg_bdl = value
        
    @property
    def burst_period_len (self) -> int:
        return (self.regset.cfg_bln)

    @setter
    def burst_period_len (self, value: int):
        # TODO check range
        self.regset.cfg_bln = value